# Part VIII — Frontend Integration (Modern Django in Real Teams)  
## 34. Django + HTMX (Server‑Rendered Interactivity That Scales)

HTMX lets you build “SPA-like” interactivity **without** moving your whole app to
React/Vue. You keep Django templates, forms, permissions, and CSRF—then add small
HTML attributes to make parts of the page update dynamically.

This chapter teaches HTMX the way it’s used in industry:

- progressive enhancement (works without JS where possible)
- partial templates (“components”)
- server-driven UI updates (HTML fragments)
- clean URL design + history
- correct CSRF handling
- strong patterns for validation errors
- testability (yes, you can test HTMX endpoints)

We’ll add a concrete feature to your **Tasks** app: **inline task status updates**
on the task list page using HTMX, including permission checks and event logging.

---

## 34.0 Learning Outcomes

By the end, you should be able to:

1. Explain where HTMX fits (vs Django-only vs SPA).
2. Add HTMX to a Django project and keep it maintainable.
3. Implement progressive enhancement:
   - normal full page requests return full templates
   - HTMX requests return fragments
4. Build partial templates and swap them into the page.
5. Handle CSRF correctly with HTMX POSTs.
6. Implement form validation with partial re-rendering (errors preserved).
7. Use key HTMX attributes:
   - `hx-get`, `hx-post`
   - `hx-target`, `hx-swap`
   - `hx-trigger`
   - `hx-push-url`
8. Use server-triggered client events (`HX-Trigger`) for toasts/messages.
9. Write tests for HTMX endpoints using the `HX-Request` header.

---

## 34.1 What HTMX Is (and Why Django Teams Like It)

### 34.1.1 The core idea
Instead of:
- a JS SPA fetching JSON and rendering components client-side,

HTMX does:
- browser sends an HTTP request (GET/POST) when an event happens (click, change)
- server returns **HTML fragment**
- HTMX swaps that HTML into the page

You keep Django as the “source of truth” for rendering.

### 34.1.2 Why HTMX is popular in Django teams
- Reuses templates, forms, permissions, CSRF
- Smaller frontend complexity than SPAs
- Fast iteration for internal tools and CRUD apps
- Great performance if you keep fragments small and querysets optimized

### 34.1.3 When HTMX is a bad fit
- highly interactive client-side state apps (complex drag/drop, offline-first)
- heavy real-time UI requiring lots of client logic (though you can combine with WebSockets)
- apps requiring sophisticated frontend architecture/design system integration

---

## 34.2 Install HTMX (Two Ways)

HTMX is just a small JS library. You can:

### Option A: Use a CDN (fastest for learning)
Add to your `templates/base.html` near end of `<body>`:

```django
<script
  src="https://unpkg.com/htmx.org@1.9.12"
  integrity=""
  crossorigin="anonymous"
></script>
```

### Option B (more production-friendly): serve it as a static file
Download `htmx.min.js` and place in:

```text
static/vendor/htmx.min.js
```

Then in `base.html`:

```django
{% load static %}
<script src="{% static 'vendor/htmx.min.js' %}"></script>
```

**Industry note:** Many teams pin the file in repo or via asset pipeline for
stability and CSP compatibility.

---

## 34.3 (Recommended) Add `django-htmx` Helper Middleware

You can use HTMX without it, but `django-htmx` makes common patterns cleaner:

- `request.htmx` boolean
- helpers for redirects and triggers

Install:

```bash
python -m pip install django-htmx
python -m pip freeze > requirements.txt
```

Add middleware in `config/settings.py` (or `base.py`) **after** SessionMiddleware and
before CommonMiddleware is typical:

```python
MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "config.middleware.RequestIdMiddleware",
    "config.access_log_middleware.AccessLogMiddleware",
    "django.contrib.sessions.middleware.SessionMiddleware",
    "django_htmx.middleware.HtmxMiddleware",
    "django.middleware.common.CommonMiddleware",
    "django.middleware.csrf.CsrfViewMiddleware",
    "django.contrib.auth.middleware.AuthenticationMiddleware",
    "django.contrib.messages.middleware.MessageMiddleware",
    "django.middleware.clickjacking.XFrameOptionsMiddleware",
]
```

### 34.3.1 If you don’t want django-htmx
You can detect HTMX requests by header:

```python
is_htmx = request.headers.get("HX-Request") == "true"
```

`django-htmx` just standardizes this into `request.htmx`.

---

## 34.4 HTMX Basics You Must Know (with concrete examples)

### 34.4.1 `hx-get` / `hx-post`
- `hx-get="/path/"` makes a GET request when triggered
- `hx-post="/path/"` makes a POST request when triggered

Example:

```html
<button hx-get="/ping/" hx-target="#result">Ping</button>
<div id="result"></div>
```

Server returns HTML fragment and HTMX swaps it into `#result`.

### 34.4.2 `hx-target` (where to insert the response)
- `hx-target="#id"` chooses the element to swap content into

### 34.4.3 `hx-swap` (how to swap)
Common swaps:
- `innerHTML` (default): replace inside target
- `outerHTML`: replace the target element itself
- `beforeend`: append at end
- `afterbegin`: prepend

Example replacing a row:

```html
<div id="task-123" hx-swap="outerHTML"> ... </div>
```

### 34.4.4 `hx-trigger`
Controls which event fires the request:
- `click` (default for buttons)
- `change` for selects
- `keyup changed delay:300ms` for search boxes

Example:

```html
<input hx-get="/search/" hx-trigger="keyup changed delay:300ms" />
```

### 34.4.5 `hx-push-url` (browser history)
If a partial update should update the URL (like filters), use:

```html
<div hx-get="/articles/?q=django" hx-push-url="true"></div>
```

This makes back/forward work more naturally.

---

## 34.5 The Professional Pattern: Full Page + Fragment (Progressive Enhancement)

A common production pattern:

- If normal request → render full page template
- If HTMX request → render only the fragment

Example:

```python
if request.htmx:
    return render(request, "tasks/partials/_task_table.html", context)
return render(request, "tasks/list.html", context)
```

Why this is industry standard:
- the page still works without HTMX
- you can reuse the same view for both experiences
- SEO and accessibility are generally better than SPA-only

---

# 34.6 Feature Build: Inline Task Status Updates with HTMX

We will implement:

- On `/orgs/<org_slug>/tasks/` list page:
  - each task row has a `<select>` for status
  - changing it sends an HTMX POST to update that task
  - server returns updated row HTML
  - row swaps instantly without full page reload
  - permissions enforced (admin or creator/assignee)

We will:
1) add a tiny form class  
2) add a view endpoint for status update  
3) add partial templates for a task row  
4) integrate HTMX attributes in list template  
5) add a server-triggered toast via `HX-Trigger`  
6) add tests

---

## 34.6.1 Add a small “status only” form

Create `tasks/forms_htmx.py`:

```python
from __future__ import annotations

from django import forms

from tasks.models import Task


class TaskStatusForm(forms.ModelForm):
    class Meta:
        model = Task
        fields = ["status"]
```

Why a ModelForm even for one field?
- it validates choices correctly
- it gives you consistent error handling
- it’s easy to render or inspect errors in templates

---

## 34.6.2 Add a service function (keep domain logic consistent)

In `tasks/services.py`, add a function that:
- sets status
- updates updated_by
- logs TaskEvent and audit log (if you implemented those)
- runs in a transaction

```python
from __future__ import annotations

from django.db import transaction

from tasks.models import Task, TaskEvent


@transaction.atomic
def set_task_status(*, task: Task, status: str, actor) -> Task:
    old_status = task.status
    if old_status == status:
        return task

    task.status = status
    task.updated_by = actor
    task.save(update_fields=["status", "updated_by", "updated_at"])

    TaskEvent.objects.create(
        task=task,
        actor=actor,
        action="status_changed",
        details={"from": old_status, "to": status},
    )

    # Optional audit log:
    try:
        from audit.services import log_event

        log_event(
            actor=actor,
            action="task.status_changed",
            target=task,
            details={"from": old_status, "to": status},
        )
    except Exception:
        # Don’t break the main action if audit log fails in dev
        pass

    return task
```

---

## 34.6.3 Add the HTMX view endpoint

Create `tasks/views_htmx.py`:

```python
from __future__ import annotations

from django.core.exceptions import PermissionDenied
from django.shortcuts import get_object_or_404, render
from django.views.decorators.http import require_POST

from orgs.services import get_membership, get_org_for_user_or_404
from tasks.forms_htmx import TaskStatusForm
from tasks.models import Task
from tasks.permissions import can_edit_task
from tasks.services import set_task_status


@require_POST
def task_status_update(request, org_slug: str, task_id: int):
    if not request.user.is_authenticated:
        raise PermissionDenied

    org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)
    membership = get_membership(user=request.user, organization=org)

    task = get_object_or_404(Task, id=task_id, organization=org)

    if not can_edit_task(membership=membership, user=request.user, task=task):
        raise PermissionDenied

    form = TaskStatusForm(request.POST, instance=task)
    if form.is_valid():
        task = set_task_status(
            task=task,
            status=form.cleaned_data["status"],
            actor=request.user,
        )

        response = render(
            request,
            "tasks/partials/_task_row.html",
            {"organization": org, "task": task},
        )

        # Server-triggered client event:
        # HTMX will fire a JS event named "toast" with this JSON detail.
        response["HX-Trigger"] = (
            '{"toast":{"level":"success","message":"Task status updated."}}'
        )
        return response

    # Invalid: return the row with errors (status select errors)
    # We render a different partial showing inline errors.
    response = render(
        request,
        "tasks/partials/_task_row_status_error.html",
        {"organization": org, "task": task, "form": form},
        status=400,
    )
    response["HX-Trigger"] = (
        '{"toast":{"level":"error","message":"Invalid status."}}'
    )
    return response
```

### Why we return HTML (not JSON)
HTMX expects HTML fragments as the swap content. That’s its whole model.

### Why return 400 on validation error
Semantically correct, but note:
- HTMX treats 4xx as “error responses” and triggers error events.
- It still swaps content by default in many setups, but behavior can vary.

If you want the simplest behavior, you can return 200 even with errors. Many teams
do that for HTMX forms. Choose and be consistent.

For this workbook:
- returning 400 is fine as long as you handle it consistently
- if you find HTMX doesn’t swap on 400 in your setup, change status to 200 for the
  invalid case.

---

## 34.6.4 Wire the URL

In `tasks/urls.py` add:

```python
from tasks import views_htmx

urlpatterns += [
    path(
        "<int:task_id>/status/",
        views_htmx.task_status_update,
        name="status_update",
    ),
]
```

Because tasks URLs are already org-scoped by include, the full path becomes:

- `/orgs/<org_slug>/tasks/<task_id>/status/`

---

## 34.6.5 Create partial templates for the task row

### `tasks/templates/tasks/partials/_task_row.html`
```django
<li id="task-{{ task.id }}">
  <a href="{% url 'tasks:detail' org_slug=organization.slug task_id=task.id %}">
    {{ task.title }}
  </a>

  —
  <code>{{ task.status }}</code>

  <form
    method="post"
    hx-post="{% url 'tasks:status_update' org_slug=organization.slug task_id=task.id %}"
    hx-trigger="change"
    hx-target="#task-{{ task.id }}"
    hx-swap="outerHTML"
    style="display:inline"
  >
    {% csrf_token %}
    <label for="id_status_{{ task.id }}" style="display:none">Status</label>

    <select name="status" id="id_status_{{ task.id }}">
      {% for value,label in task.Status.choices %}
        <option value="{{ value }}" {% if task.status == value %}selected{% endif %}>
          {{ label }}
        </option>
      {% endfor %}
    </select>
  </form>

  {% if task.assigned_to %}
    — assigned to <strong>{{ task.assigned_to.username }}</strong>
  {% endif %}
</li>
```

#### Important: why the form uses `hx-trigger="change"`
HTMX will submit this form when the `<select>` changes.

Why include `{% csrf_token %}`?
- It’s a POST.
- With session-based auth, CSRF protection applies.
- HTMX will include the hidden CSRF input in the request body, so Django accepts it.

### Error partial (optional but clean): `_task_row_status_error.html`

```django
<li id="task-{{ task.id }}" style="background:#fff3f3; border:1px solid #f2b6b6;">
  <strong>{{ task.title }}</strong>
  <div>
    Could not update status:
    {% for err in form.status.errors %}
      <span class="error">{{ err }}</span>
    {% endfor %}
  </div>

  {# Re-render the normal row form so user can try again #}
  {% include "tasks/partials/_task_row.html" with organization=organization task=task %}
</li>
```

---

## 34.6.6 Update the tasks list template to use the partial

In `tasks/templates/tasks/list.html`, replace your loop with:

```django
<ul>
  {% for t in tasks %}
    {% include "tasks/partials/_task_row.html" with organization=organization task=t %}
  {% empty %}
    <li>No tasks found.</li>
  {% endfor %}
</ul>
```

Now every row has HTMX-enabled status updates.

---

## 34.6.7 Add a simple toast handler for `HX-Trigger`

Add in `templates/base.html` (after HTMX script):

```django
<script>
  document.body.addEventListener("toast", function (evt) {
    const detail = evt.detail || {};
    const msg = detail.message || "Notification";
    const level = detail.level || "info";

    // Minimal UI: alert.
    // In real apps, render a styled toast element.
    console.log("toast:", level, msg);
  });
</script>
```

### Why `HX-Trigger` is useful
Your server decides when to show UI feedback, without inventing a separate “JSON
status protocol.”

This keeps the UI server-driven and consistent with Django messages patterns.

---

# 34.7 HTMX + Validation Patterns (Forms Done Correctly)

### Pattern A: Swap the form fragment with errors
- HTMX POST submits the form
- server returns the same form HTML with error messages
- HTMX swaps it in place

This is identical to classic Django form handling, just partial.

### Pattern B: PRG for success, inline render for errors
- On success: return `HX-Redirect` (HTMX special header) or normal redirect.
- On invalid: return fragment with errors.

If you use `django-htmx`, you can do:

```python
from django_htmx.http import HttpResponseClientRedirect

return HttpResponseClientRedirect(success_url)
```

This tells HTMX to do a client-side redirect properly.

---

# 34.8 HTMX + Caching (Be careful)

Don’t cache HTMX fragments that include:
- user-specific data
- CSRF tokens
- personalized permissions

If you cache, vary by:
- user/session
- query params
- relevant headers

Rule of thumb:
- cache public fragments (article list sidebar “top tags”) with short TTL
- don’t cache authenticated fragments unless you know exactly what you’re doing

---

# 34.9 Testing HTMX Endpoints (Yes, It’s Testable)

HTMX requests include headers like:
- `HX-Request: true`

Django test client can send that header.

Create `tasks/tests_htmx.py`:

```python
from django.test import TestCase
from django.urls import reverse

from orgs.models import Membership
from tests.factories import MembershipFactory, OrganizationFactory, TaskFactory, UserFactory


class TaskHtmxTests(TestCase):
    def setUp(self):
        self.org = OrganizationFactory(slug="acme")
        self.user = UserFactory(username="member")
        MembershipFactory(
            organization=self.org,
            user=self.user,
            role=Membership.Role.ADMIN,
        )
        self.task = TaskFactory(organization=self.org, created_by=self.user, updated_by=self.user)

        self.client.login(username="member", password="pass12345")

    def test_status_update_returns_row_fragment(self):
        url = reverse(
            "tasks:status_update",
            kwargs={"org_slug": "acme", "task_id": self.task.id},
        )

        resp = self.client.post(
            url,
            {"status": "done"},
            HTTP_HX_REQUEST="true",
        )

        self.assertEqual(resp.status_code, 200)
        self.assertIn(f'id="task-{self.task.id}"', resp.content.decode("utf-8"))
        self.assertEqual(resp.headers.get("HX-Trigger") is not None, True)
```

### Why this test is meaningful
- It proves the endpoint works for HTMX requests.
- It proves response is an HTML fragment containing the correct row id.
- It validates the server-trigger event header exists.

---

# 34.10 Exercises (Do These Before Moving On)

1. **Convert task list filters to HTMX**
   - Make the filter form submit via `hx-get`
   - Target the task list `<ul>` or a `<div>` wrapper
   - Preserve URL using `hx-push-url="true"`
   - Return a fragment for HTMX requests and full page for normal requests

2. **Convert comment submission to HTMX**
   - POST comment form via HTMX
   - On success, swap the comment form area to a “submitted for review” message
   - On invalid, swap in the form with errors

3. **Add optimistic UI**
   - While HTMX request is running, show a “Saving…” indicator
   - Use `hx-indicator="#saving"` and a hidden element

4. **Permission regression**
   - Write a test that a non-member cannot status-update (expect 404)
   - Write a test that an unauthorized member gets 403

---

## 34.11 Chapter Summary

- HTMX adds interactivity by swapping server-rendered HTML fragments into a page.
- The industry-standard pattern is progressive enhancement:
  - full pages for normal requests
  - fragments for HTMX requests
- Use Django forms + CSRF normally—HTMX fits the Django security model naturally.
- Keep architecture clean:
  - views stay thin
  - services handle workflows
  - partial templates act like components
- Use `HX-Trigger` for server-driven UI events (toasts/messages).
- Test HTMX endpoints by sending `HX-Request` headers.

---

Next chapter: **35. Django + SPA / Frontend Frameworks (React/Vue/Next)**  
We’ll cover architecture options (monolith vs split), auth patterns (cookies vs
tokens), CORS/CSRF for SPAs, deployment topology, and best practices for DRF-backed
frontends in real teams.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../7. Data_integrations_and_advanced_orm/33. search_files_and_external_services.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='35. django_plus_frontend_frameworks.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
